In [ ]:
import scanpy as sc
import pandas as pd
import os
import anndata as ad
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import squidpy as sq
from wrapper_functions import *
sns.set()

In [ ]:
# Automatically re-load wrapper functions after an update
# Find details here: https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [ ]:
sc.logging.print_versions()
sc.set_figure_params(facecolor="white", figsize=(6, 6))
sc.settings.verbosity = 3

In [ ]:
organism = Organism.mouse
analyze_params = Analyze(protocol=Protocol.FF, organism=organism)

In [ ]:
root_path = os.getcwd()
inpath='your_inpath_folder' # Replace with the location of your samples
results_folder = os.path.join(root_path, 'analyzed/zonation')

In [ ]:
file_names = [f for f in os.listdir(results_folder) if os.path.isfile(os.path.join(results_folder, f))]

adata_list = [ad.read(os.path.join(results_folder, file)) for file in file_names if file.endswith('.h5ad')]

In [ ]:
adata_list_control = []

for current_adata in adata_list:
    
    if current_adata.obs['Condition'].unique()[0] == "Untreated":
         adata_list_control.append(current_adata)

We concatenate to normalize together and make everything comparable

In [ ]:
adata_concat = sc.concat(
    adata_list_control,
    label="library_id",
    uns_merge="unique",
    keys=[
        k
        for d in [adata.uns["spatial"] for adata in adata_list_control]
        for k, v in d.items()
    ],
    index_unique="-",
    join='outer',
)

In [ ]:
adata_concat_norm = adata_concat.copy()
adata_concat_norm.raw = adata_concat.copy()
sc.pp.normalize_total(adata_concat_norm)
sc.pp.log1p(adata_concat_norm)
sc.pp.highly_variable_genes(adata_concat_norm,  flavor="seurat")

In [ ]:
genes_positive = [
    "AU040320", "Ndst1", "Tm9sf2", "B3galt6", "Arf1", "Itgav", "Itga5", 
    "Itgb1", "Itgb5", "Rpsa", "Pdgfra", "Cd9", "Fgfr1", "B4galt1", 
    "Arhgap26", "Cdc42", "Dnm1", "Met", "Egfr", "B3gat3", "Slc35b2", 
    "Ext1", "Ext2", "Cog7", "Rab7a", "Stx5", "Rab11a", "Vps35l", 
    "Vps53", "Rab6a", "Rgp1", "Atp2c1", "Gpr108", "Atp6v0a2", 
    "Rnf121", "Kpnb1"
]
common_genes_positive = [gene for gene in genes_positive if gene in adata_concat_norm.var.index]

In [ ]:
genes_negative = [
    "Ube2i", "Uba2", "Sae1",
    "Suv4-20h1", "Chaf1a", "Npat", "Casp8ap2", "Phc3", "Phf5a", 
    "Sf3b2", "U2af1", "Zfp638", "Fkbp4", "Mre11a", "Rad50", 
    "Nbn", "Ifnar2", "Wwc2", "Adgra2", "Rtbdn", "Troap", "Sox15", 
    "Slc5a2", "Clic2", "Grpr"
]
common_genes_negative = [gene for gene in genes_negative if gene in adata_concat_norm.var.index]

In [ ]:
sc.pl.matrixplot(adata_concat_norm, common_genes_positive, groupby=['Sample_ID','zonation'], dendrogram=False, use_raw=False)

In [ ]:
sc.pl.matrixplot(adata_concat_norm, common_genes_positive, groupby=['Gender','zonation'], dendrogram=False, use_raw=False)

In [ ]:
sc.pl.matrixplot(adata_concat_norm, common_genes_negative, groupby=['Sample_ID','zonation'], dendrogram=False, use_raw=False)

In [ ]:
sc.pl.matrixplot(adata_concat_norm, common_genes_negative, groupby=['Gender','zonation'], dendrogram=False, use_raw=False)

In [ ]:
sc.pl.stacked_violin(adata_concat_norm, common_genes_negative, groupby=['zonation','Gender'], dendrogram=True)

In [ ]:
! jupyter nbconvert --to html 25_ControlSamples_GeneExpressionExploration.ipynb